In [1]:
import pandas as pd
import nltk as nk
import numpy as np
from sklearn.feature_extraction import stop_words
from collections import Counter
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data1 = pd.read_csv("ride_data1.csv")
data2 = pd.read_csv("ride_data2.csv")

In [3]:
data = pd.concat([data1,data2])

In [4]:
data.head()

,ride_review,ride_rating,sentiment
0,I had just completed running the New York Mara...,1.0,0
1,My appointment time for auto repairs required ...,1.0,0
2,Whether I am using Uber for a ride service or ...,1.0,0
3,Why is it so hard for you to understand that i...,1.0,0
4,"I was in South Beach, FL. I was staying at a m...",1.0,0


## Data Preprocessing

In [5]:
stopwords = stopwords.words('english')
with open("stopwords_stanford") as f:
    stanford_sw = f.readlines()

In [6]:
stanford_sw = [x.strip() for x in stanford_sw]

In [7]:
sw = list(set(stopwords + stanford_sw + list(stop_words.ENGLISH_STOP_WORDS)))
sw = [x.lower() for x in sw]

In [8]:
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.tokenize import sent_tokenize,word_tokenize

In [11]:
def data_preprocessing(text):
    sb = SnowballStemmer("english")
    for i in range(0,len(text['ride_review'])):
        tokens = word_tokenize(text['ride_review'].iloc[i])
        tokens = [x.lower() for x in tokens if x.lower() not in sw]
        tokens = list(filter(lambda x:x.isalpha(),tokens))
        stem_tokens = [sb.stem(x) for x in tokens]
        c = Counter()
        c.update(stem_tokens)
        d = dict(c)
        seen = set()
        result = []
        for item in tokens:
            if sb.stem(item) not in seen:
                seen.add(sb.stem(item))
                result.append(item)
        review = ' '.join(x for x in result)
        text['ride_review'].iloc[i] = review
    return text

In [12]:
pro_data = data_preprocessing(data)

/Users/purvank/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
pro_data['ride_review'].iloc[0]

'completed new requested let tell started right driver agreed pick drop locations called spoke know actually blocked pickup away race road got notification rider joining spoken shortly passenger went wrong moments continuing destination phone asked going claimed need confused far time clearly played dumb saying understand works middle strange wanting argue lucrative jerk peacefully exited stood private residence shivering cold degree rainy weather dark mile safe seek rest warmth safety sooner scrawling fare piece paper standing outside meet inkling conscientiousness nerve charge partial excited using taxis coming lyft connected minutes long warm dressed clothes weary expecting stranded pace wait main traffic heavy rode closures able realistic eternal happiness arrived delivered nicely day finding way file complaint sent couple hours replied wet blanket apology inconvenience offered credit make worse taken wrote corrected harm previous good reputation earned pay preserve relationships b

In [14]:
#n_data.to_csv("cv_data.csv",index=False)

In [15]:
#f = open("example.txt", "w")
#f.write("\n".join(map(lambda x: str(x), l)))
#f.close()

In [35]:
tf_idf = TfidfVectorizer(ngram_range=(1,3),min_df=5)

In [36]:
d1 = tf_idf.fit_transform(pro_data["ride_review"]).toarray()

In [37]:
n_data1 = pd.DataFrame(d1,columns=tf_idf.get_feature_names())

In [38]:
sb = SnowballStemmer("english")

In [39]:
stem_tokens = [sb.stem(x) for x in tf_idf.get_feature_names()]
c = Counter()
c.update(stem_tokens)
d = dict(c)
deleted = []
print (type(tf_idf.get_feature_names()))

<class 'list'>


In [40]:
a = list(tf_idf.get_feature_names())

In [41]:
seen = set()
result = []
for item in a:
    if sb.stem(item) not in seen:
        seen.add(sb.stem(item))
        result.append(item)

In [42]:
n_data2 = n_data1[result]

In [43]:
n_data2.shape

(1344, 1563)

In [44]:
n_data2['sentiment'] = pro_data['sentiment'].values

/Users/purvank/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [45]:
n_data2.shape

(1344, 1564)

In [46]:
#n_data1['sentiment'] = pro_data['sentiment'].values

In [47]:
n_data2.to_csv("data2.csv",index=False)